# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd 
from sqlalchemy import create_engine 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import re
import pickle

In [2]:
# load data from database
db_path = "../data/DisasterResponse.db"
engine = create_engine('sqlite:///'+ db_path )
df = pd.read_sql_table('Messages', engine)


In [3]:
df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#to check classes values in each column
for column in df:
    print(df[column].unique())

[    2     7     8 ... 30263 30264 30265]
['Weather update - a cold front from Cuba that could pass over Haiti'
 'Is the Hurricane over or is it not over'
 'Looking for someone but no name' ...
 "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families."
 'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.'
 'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.']
['Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi'
 'Cyclone nan fini osinon li pa fini'
 'Patnm, di Maryani relem pou li banm nouvel li ak timoun

In [5]:
#check column name where it contains 0 only.
df.iloc[:, 12]

0        0
1        0
2        0
3        0
4        0
        ..
26211    0
26212    0
26213    0
26214    1
26215    0
Name: military, Length: 26216, dtype: int64

It can be seen for the 36 columns all of their classes are either 0 or 1 except for: 
- related column contains 0 , 1 , 2 
- child_alone column contains 0 only

In [6]:
#drop child_alone column as it's won't be beneficial in the classification problem
df.drop('child_alone', axis='columns',inplace=True)

In [7]:
#check each unique value count for the related columns 
df['related'].groupby(df['related']).count()

related
0     6122
1    19906
2      188
Name: related, dtype: int64

As it's shown 188 rows contain the value 2 for the related column
Will replace it with 0 to help with the imbalancing 

In [8]:
#replace the value of 2 with 0 
df['related'] = df['related'].replace(2,0)

In [9]:
#check each unique value count for the related columns, the replacement worked
df['related'].groupby(df['related']).count()

related
0     6310
1    19906
Name: related, dtype: int64

In [10]:
#spilt the data into variables and targets
X = df.iloc[:, 1]
y = df.iloc[:,4:]

In [11]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    """
    This function takes the messages normalize,tokenize,remove stop words and lemmatize them.

    Arguments:
        text:the messages.

    Returns: the cleaned tokens.
        
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    words = word_tokenize(text)
    #remove stop words 
    words = [w for w in text if w not in stopwords.words("english")]
    # Reduce words to their root form
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

  
    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
#create Multi Output Classifier using random forest
clf = MultiOutputClassifier(GradientBoostingClassifier())
#create a pipeline
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())]))
            ])),
        ('clf', clf)
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
#Split the data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=109)

In [15]:
#train pipeline
model = pipeline.fit(X_train, y_train)


In [28]:
water = ["fire"]
water_pred = model.predict(water)
water_pred

array([[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0]])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
#predict on test dataset
y_pred = model.predict(X_test)

#Evaluate results
#print(classification_report(y_test, y_pred, target_names=y_test.columns))


In [26]:
i = 0
for col in y_test:
    print(classification_report(y_test[col], y_pred[:, i]))

              precision    recall  f1-score   support

           0       0.60      0.14      0.22      1591
           1       0.78      0.97      0.86      4963

    accuracy                           0.77      6554
   macro avg       0.69      0.55      0.54      6554
weighted avg       0.73      0.77      0.71      6554

              precision    recall  f1-score   support

           0       0.91      0.06      0.11      5400
           1       0.18      0.97      0.31      1154

    accuracy                           0.22      6554
   macro avg       0.55      0.52      0.21      6554
weighted avg       0.78      0.22      0.15      6554

              precision    recall  f1-score   support

           0       1.00      0.05      0.10      6528
           1       0.00      0.96      0.01        26

    accuracy                           0.06      6554
   macro avg       0.50      0.51      0.06      6554
weighted avg       0.99      0.06      0.10      6554

              preci

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    
        'clf__estimator__n_estimators': [50,100]    
    }


cv = GridSearchCV(pipeline, parameters , cv = 3 )
model = cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_improved = model.predict(X_test)

print(classification_report(y_test, y_pred_improved , target_names = y.columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#creating classifier 
clf2 = MultiOutputClassifier(RandomForestClassifier())
#create a pipeline
pipeline2 = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())]))
            ])),
        ('clf', clf2)
    ])

In [ ]:
#train model
pipeline2.fit(X_train, y_train)

#predict on test dataset
y_pred2 = pipeline2.predict(X_test)

#Evaluate results
print(classification_report(y_test, y_pred2, target_names=y_test.columns))

### 9. Export your model as a pickle file

In [ ]:
#export model
pickle.dump(cv, open('classifier.pkl', 'wb'))

In [ ]:
from platform import python_version

print(python_version())

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.